# Open Information Extraction

Information extraction takes a body of freeform text and extracts the contained information in a computer interpretable form. The word *open* simply means that the text/facts are arbitrary, so it will work with any input rather than a specific domain (e.g. legal texts).

As an example, given the input:

> "Trolls really don't like the sun."
  

you may extract the "fact":
```
('Trolls', 'do not like', 'the sun')
```

The approach is based on the paper "*Identifying Relations for Open Information Extraction*", by Fader, Soderland & Etzioni. Parts have been updated with more recent, or more ML, techniques however (Q3 and Q4 match the paper; Q1 and Q2 don't). You don't have to read the paper as this workbook takes you through the process; this lab is also intended as a tutorial/opportunity to see one way such a system works. Some parts are rule based, as that's still often the case. Note that the several parts of a complete system have been dropped, as the lab would be too much work otherwise; consequentially it isn't going to work that well.

The steps of the system are as follows:
*  Tokenise and split on sentences *(provided)*


1. Part of speech tagging - token level
2. Part of speech tagging - sentence level
3. Named entity resolution
4. Relation extraction


*  Summarise "*20,000 leagues under the seas*" by Jules Verne *(provided)*

## Ogonek

A tiny NLP library

### Tokenisation and sentence splitting
`ogonek.Tokenise()`

A class that tokenises some text and splits it into sentences. Construct an instance with `tokens = ogonek.Tokenise('My text')`; it then has the same interface as a list of lists:
* `len(tokens)`: Number of extracted sentences (not words)
* `tokens[i]`: Sentence i, where i ranges from 0 to one less than `len(tokens)`. A sentence is a list of tokens.



### Word vectors
`ogonek.Glove()`

Constructing a `glove = ogonek.Glove()` object loads a heavily pruned Glove word vectors from the file `baby_glove.zip` into memory, and will then translate tokens into word vectors. Note that it automatically lowercases any token it is handed, so you don't need to. Has the following interface:
* `glove.len_vec()` - Returns the length of the word vectors; should be 300.
* `len(glove)` - Returns how many word vectors it knows of.
* `token in glove` - Returns `True` if it has a word vector for that token, `False` otherwise.
* `glove[token]` - Returns the word vector for the given token; raises an error if it does not have one.
* `glove.decode(token)` - Returns the word vector for the given token, but if the word vector is unknown returns a vector of zeros instead (silent failure).
* `glove.decodes(list of tokens)` - Returns a list of word vectors, one for each token. Has the same silent failure behaviour as `decode`.



### Groningen Meaning Bank dataset
`ogonek.GMB()`

Provides access to the Groningen Meaning Bank dataset, which is supplied in the file `ner_dataset.csv`. Replicates the interface of the tokenisation system as far as it can. Construct with `gmb = ogonek.GMB()`; has the following interface:
* `len(gmb)`: Number of sentences (not words) in data set
* `gmb[i]`: Sentence i, where i ranges from 0 to one less than `len(gmb)`. A sentence is a list of tokens.
* `gmb.pos(i)`: A list of POS tags that match with sentence i. Note that these are the full Penn Treebank tags (not the reduced set used below).
* `gmb.ner(i)`: A list of named entities that match with sentence i. Using outside-inside scheme.



### Pretty printing

`ogonek.aligned_print(*)` takes multiple lists and prints them out, aligning them so that all elements in position 0 of all lists are aligned vertically (extra space added as required), and then elements in position 1 and so on. For showing tags and a sentence with everything aligned. Also does word wrap and colour coding.

In [1]:
%matplotlib inline

import time
import string
import re

import numpy
import matplotlib.pyplot as plt

import ogonek

## Useful variables

In [2]:
# Dictionary giving descriptions of the reduced part of speech tags...
rpos_desc = {'C' : 'Coordinating conjunction',
             '0' : 'Cardinal number',
             'D' : 'Determiner',
             'E' : 'Existential there',
             'I' : 'Preposition or subordinating conjunction',
             'J' : 'Adjective',
             'N' : 'Noun',
             'P' : 'Predeterminer',
             'S' : 'Possessive ending',
             'M' : 'Pronoun',
             'R' : 'Adverb',
             'Z' : 'Particle',
             'T' : 'to',
             'V' : 'Verb',
             'A' : 'Anything else',
             '.' : 'All punctuation'}


# Reduced list of part of speech tags as a list...
num_to_rpos = ['C', '0', 'D', 'E', 'I', 'J', 'N', 'P',
               'S', 'M', 'R', 'Z', 'T', 'V', 'A', '.']


# Dictionary that maps a reduced part of speech
# tag to it's index in the above list; useful for vectors/matrices etc...
rpos_to_num = {'C' : 0,
               '0' : 1,
               'D' : 2,
               'E' : 3,
               'I' : 4,
               'J' : 5,
               'N' : 6,
               'P' : 7,
               'S' : 8,
               'M' : 9,
               'R' : 10,
               'Z' : 11,
               'T' : 12,
               'V' : 13,
               'A' : 14,
               '.' : 15}


# Dictionary that maps the full part of speech tags to the reduced set...
pos_to_rpos = {'CC' : 'C',
               'CD' : '0',
               'DT' : 'D',
               'EX' : 'E',
               'FW' : 'A',
               'IN' : 'I',
               'JJ' : 'J',
               'JJR' : 'J',
               'JJS' : 'J',
               'LS' : 'A',
               'MD' : 'A',
               'NN' : 'N',
               'NNS' : 'N',
               'NNP' : 'N',
               'NNPS' : 'N',
               'PDT' : 'P',
               'POS' : 'S',
               'PRP' : 'M',
               'PRP$' : 'M',
               'RB' : 'R',
               'RBR' : 'R',
               'RBS' : 'R',
               'RP' : 'Z',
               'SYM' : 'A',
               'TO' : 'T',
               'UH' : 'A',
               'VB' : 'V',
               'VBD' : 'V',
               'VBG' : 'V',
               'VBN' : 'V',
               'VBP' : 'V',
               'VBZ' : 'V',
               'WDT' : 'D',
               'WP' : 'M',
               'WP$' : 'S',
               'WRB' : 'R',
               '-' : '.',
               'LRB' : '.',
               'RRB' : '.',
               '``' : '.',
               '"' : '.',
               '.' : '.',
               ',' : '.',
               ';' : '.',
               ':' : '.',
               '$' : '.'}

## Load book, tokenise and split on sentences
The below code reads in the book, chops it down to just the text of the book, and then tokenises it using the provided `ogonek` library.

In [3]:
# Loop file, only keeping lines between indicators...
lines = []
record = False

with open('20,000 Leagues Under the Seas.txt', 'r', encoding='utf8') as fin:
    for line in fin:
        if record:
            if line.startswith('***END OF THE PROJECT GUTENBERG'):
                break
            lines.append(line)
    
        else:
            if line.startswith('***START OF THE PROJECT GUTENBERG'):
                record = True

text = ''.join(lines)

# Tokenise...
under_the_seas = ogonek.Tokenise(text)

# Print 10 random sentences to check it worked...
numpy.random.seed(0)

for i in range(10):
    toks = numpy.random.choice(under_the_seas)
    print('{:02d}. {}'.format(i+1, ' '.join(toks)))


01. It was the regime of verticality .
02. Now then , the tides are not strong in the Pacific , and if you can not unballast the Nautilus , which seems impossible to me , I do not see how it will float off . "
03. Captain Nemo left the cave , and we climbed back up the bank of shellfish in the midst of these clear waters not yet disturbed by divers at work .
04. Likewise the pilothouse and the beacon housing were withdrawn into the hull until they lay exactly flush with it .
05. Instead of digging all around the Nautilus , which would have entailed even greater difficulties , Captain Nemo had an immense trench outlined on the ice , eight meters from our port quarter .
06. We would not go five miles without bumping into a fellow countryman .
07. The oars , mast , and sail are in the skiff .
08. Under existing conditions some ten men at the most should be enough to operate it . "
09. Nobody appeared on our arrival .
10. We gasped .


# 1. Part of speech tagging - token level

The goal here is to train a classifier that indicates which of the part of speech tags (the reduced set provided above) each word is. For this initial approach you're going to treat words (tokens) individually, without context. For features the Glove word vectors are going to be used (provided by `ogonek.Glove()`).

Instead of training a single classifier a slight modification of a random kitchen sink for each part of speech tag is going to be used. Specifically, a logistic random kitchen sink that indicates the probability that the word should be labelled with the associated tag. This is a *one vs all* classifier - you have a classifier for every tag, run them all on each word, and then select the tag with the highest probability (it's inconsistent - they won't sum to 1!). A logistic random kitchen sink is simply a normal kitchen sink that is pushed through a sigmoid function (in neural network terms, the final layer has a non-linearity),
$$\operatorname{Sig}(z) = \frac{1}{1 + e^{-z}}$$
such that the final binary classifier is
$$P(\textrm{tag}) = \operatorname{Sig}\left(\sum_{k \in K} \alpha_k \phi\left(\vec{x} \cdot \vec{w}_k\right)\right)$$
For the cost function you should maximise the log likelihood of the dataset. This will require gradient descent; Nestorov or better, including backtracking line search to select the initial step size. You can either differentiate yourself, copy the equations from lecture 7 of ML1 (where they were derived), or use tensor flow; your choice! The non-linearity, $\phi(\cdot)$ is up to you ($\sin$ works). Remember to include the original features plus the value `1` when creating the extended feature vector (so it has a bias term). It is suggested to use 300 random features, in addition to the 300 provided by glove (total of 601 - bias term is the +1), as that keeps the resulting data matrix during training small enough that it completes reasonably quickly.

The Groningen Meaning Bank dataset has been provided; it can be accessed via the class `ogonek.GMB`. It includes lots of sentences, each as a list of tokens, plus part of speech tags as a list aligned with the sentence.
Source: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [4]:
a = (numpy.random.rand(1,10)) #create a random (1,10) array
print ("a: \n" + str(a))
result = (numpy.where(a>0.5,1,0)) 
print ("result: \n" +str(result))

a: 
[[0.79172504 0.52889492 0.56804456 0.92559664 0.07103606 0.0871293
  0.0202184  0.83261985 0.77815675 0.87001215]]
result: 
[[1 1 1 1 0 0 0 1 1 1]]


In [5]:
# Load word vectors; in a seperate cell as this takes a couple seconds...
glove = ogonek.Glove()

# Groningen Meaning Bank dataset - a set of sentences each tagged
# with part of speech and named entity recognitiuon tags...
gmb = ogonek.GMB()
print('GMB sentences = {}'.format(len(gmb)))
print()

# Print out 5 random sentences from GMB with POS and NER tags, to illustrate the data...
numpy.random.seed(1)
for _ in range(5):
    i = numpy.random.randint(len(gmb))
    ogonek.aligned_print(gmb[i], gmb.pos(i), gmb.ner(i))

GMB sentences = 47959

The U.S.  space agency is  making final preparations to launch the first 
DT  NNP   NN    NN     VBZ VBG    JJ    NNS          TO VB     DT  JJ    
O   B-geo O     O      O   O      O     O            O  O      O   O     

direct space probe to the distant planet of Pluto . 
JJ     NN    NN    TO DT  JJ      NN     IN NNP   . 
O      O     O     O  O   O       O      O  B-geo O 

On Monday , the freighter Torgelow was hijacked off the eastern coast of 
IN NNP    , DT  NN        NNP      VBD VBN      IN  DT  JJ      NN    IN 
O  B-tim  O O   O         B-art    O   O        O   O   O       O     O  

Somalia . 
NNP     . 
B-geo   O 

Chile and Bolivia are associate members . 
NNP   CC  NNP     VBP JJ        NNS     . 
B-gpe O   B-gpe   O   O         O       O 

Venezuela has freed 11 Colombian soldiers who had been detained after entering 
NNP       VBZ VBN   CD JJ        NNS      WP  VBD VBN  VBN      IN    VBG      
B-geo     O   O     O  B-gpe     O        O   O

In [9]:
# A test/train split - train with [0:split], test with [split:len(gmb)]
split = int(len(gmb) * 0.3) # Have a lot of data, and don't want you waiting around too long to train!
print('Using {} sentences for training'.format(split))

x_train = gmb[0:split]
x_test = gmb[split:len(gmb)]

con_train = numpy.concatenate(x_train)
con_test = numpy.concatenate(x_test)
train_decode = glove.decodes(con_train)
test_decode = glove.decodes(con_test)

def sigmoid(z):
    return 1 / (1 + numpy.exp(-z))

# update alpha gradient to get new alpha
def Nesterov_gradient(y,ex,lamb_da, step_size,alpha_grad,initial_alpha):
    update_gradient = numpy.random.normal(0, 1, 601)
    alpha = initial_alpha
    loglike = loss_fun(y,ex,alpha)
    j = 0
    
    for i in range (256):
        
        alpha_grad = alpha_gradient(y,ex,alpha+lamb_da* update_gradient)
        
        update_gradient = lamb_da * update_gradient + step_size * alpha_grad
        alpha = alpha + update_gradient
        
        new_loglike = loss_fun(y,ex,alpha)
        # if gradient stop update, after 8 times break of loop
        if new_loglike > loglike:
            loglike = new_loglike

        else:
            j += 1
            if j>8:
                break 
        
    return alpha,loglike

# from the log likelihood to get the alpha gradient
def alpha_gradient(y,ex,alpha):    
    alpha_grad = numpy.zeros(601)

    p = sigmoid(ex.dot(alpha))
    alpha_grad += (y - p)@ ex

    return alpha_grad

# Armijo_goldstein which is backtracking line search,this is to get the best step size
# only have alpha as variable
def Armijo_goldstein(x,y,ex,eta,alpha,alpha_grad,w):
    mu = 0.5

    while (loss_fun(y,ex,alpha + eta * alpha_grad) < loss_fun(y,ex,alpha) + eta * mu * numpy.linalg.norm(alpha_gradient(y,ex,alpha), ord = 2)):       
        eta =  eta * 0.8
        if eta < 1e-6:
            break
    return eta

# loss function is calculated the log likelihood
# loss function only have alpha as variable
def loss_fun(y,ex,alpha):

    q = sigmoid(ex.dot(alpha))
    q = numpy.clip(q, 1e-3, 1 - 1e-3)
    p = y
    
    L =sum(p*numpy.log(q) + (1-p)*numpy.log(1-q))
    # maximum log likelihood value for calculation
    
    return L

def train(x, w):

    nf = numpy.sin(numpy.einsum('ef,gf->eg', x, w)) 
    ex = numpy.append(x, nf, axis=1)
    bias = numpy.ones((x.shape[0],1))
    ex = numpy.append(ex, bias, axis=1)
    return nf, ex

x_train = numpy.array(train_decode)
x_test = numpy.array(test_decode)

def train_tag_model(tag):   
    start = time.time()

    lamb_da = 0.8
    
    y_train = (numpy.array([pos_to_rpos[tag] for i in range(split) for tag in gmb.pos(i)]) == tag).astype(int)

    w = numpy.random.standard_normal((300, 300))
    # alpha size is 601
    initial_alpha = numpy.random.normal(0, 1, 601)
    initial_alpha /= numpy.linalg.norm(initial_alpha)
    
    # nf, ex
    nf, ex = train(x_train,w)
    # alpha gradient
    alpha_grad =  alpha_gradient(y_train, ex,initial_alpha)
   
    # best step size: backtracking
    eta = 1
    best_eta = Armijo_goldstein(x_train,y_train,ex,eta,initial_alpha,alpha_grad,w)    
    print('best_eta',best_eta)
    # nestrov
#     best_eta = 1
    new_alpha,loglike = Nesterov_gradient(y_train,ex,lamb_da, best_eta, alpha_grad,initial_alpha)  
    print('loglike',loglike)
    end = time.time()
    print('  (took {:g} seconds)'.format(end-start))
    return new_alpha,w

# Code to train a model for each reduced POS tag...
rpos_model = {}
for tag in rpos_desc:
    print('Training {}'.format(tag))
    rpos_model[tag] = train_tag_model(tag)


Using 14387 sentences for training
Training C


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in exp


best_eta 0.40960000000000013
loglike -2676.2416434100764
  (took 79.2629 seconds)
Training 0
best_eta 0.40960000000000013
loglike -5811.897890510623
  (took 47.7612 seconds)
Training D
best_eta 0.0001063382396627935
loglike -54965.52249305237
  (took 39.9793 seconds)
Training E
best_eta 0.40960000000000013
loglike -3663.8971543855987
  (took 70.2357 seconds)
Training I
best_eta 1.1417981541647708e-05
loglike -221703.79048375427
  (took 50.9858 seconds)
Training J
best_eta 0.13421772800000006
loglike -85641.73545240676
  (took 43.2894 seconds)
Training N
best_eta 1.2259964326927154e-06
loglike -223552.12578553427
  (took 49.1793 seconds)
Training P
best_eta 0.5120000000000001
loglike -3387.626963236436
  (took 95.9931 seconds)
Training S
best_eta 0.40960000000000013
loglike -2558.8163897880495
  (took 73.6141 seconds)
Training M
best_eta 0.40960000000000013
loglike -38612.076334565725
  (took 33.7343 seconds)
Training R
best_eta 0.32768000000000014
loglike -56265.74154886884
  (took 33.

In [12]:
def token_pos(sentence):
    """Given a sentence, as a list of tokens, this should return part of
    speech tags, as a list of strings (the codes in the rpos_desc dictionary).
    Basically calls the models for each tag and selects the tag with the
    highest probability."""
    
    x = glove.decodes(sentence)
    x = numpy.array(x)
    
    z_value = []
    predict_tag = []
    final_pred = []
    tag_pred = ['i'] * len(sentence)
    for tag in rpos_desc:
        update_alpha,weight = rpos_model[tag]

        nf = numpy.sin(numpy.einsum('ef,gf->eg', x, weight))
        ex = numpy.append(x, nf, axis=1)
        bias = numpy.ones((x.shape[0],1))
        ex = numpy.append(ex, bias, axis=1)

        pred_y = sigmoid(ex.dot(update_alpha)) # calculate the prediction value

        predict_tag.append(pred_y)
    
    kitc_sink_p = numpy.zeros(len(sentence))
    for j in range(len(sentence)):
        word_list = []

        for i in range (16):
            word_list.append(predict_tag[i][j]) 
        # find the maximum probability tag index
        index = numpy.argmax(word_list)
        
        kitc_sink_p[j] = max(word_list)
        tag_pred[j] = num_to_rpos[index]        
 
    return tag_pred,kitc_sink_p
                
#     # maximum 16 probabilities
# Code to test the performance of your POS tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 

start = time.time()
# for i in range(split, len(gmb)):
for i in range(0, split):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess,senten_prob = token_pos(gmb[i])
    truth = gmb.pos(i)
    
    for g,t in zip(guess, truth):
        if g==pos_to_rpos[t]:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in exp



Percentage correct = 63.4%
  (took 330.334 seconds)


# 2. Part of speech tagging - sentence level

While the previous step works very well you need POS tags to be super accurate, as everything else depends on them. You will now introduce context. This is done by calculating transition probabilities between tags and solving a Markov random chain using the forward-backwards algorithm (or just forward if you keep links; it's dynamic programming) to find the maximum a posteriori (MAP) POS tag assignment for the entire sentence. The adjacency matrix should contain $\log P(\textrm{second pos tag} | \textrm{first pos tag})$.

In [158]:
# adjacency probabilities
prob_matrix = numpy.zeros((16,16))

for i in range (split, len(gmb)):
    sent = gmb.pos(i)
    j = 0
    while j < len(sent)-1:
        row = rpos_to_num[pos_to_rpos[sent[j]]]
        column = rpos_to_num[pos_to_rpos[sent[j+1]]]
        j += 1
        prob_matrix[column][row] +=1
        
# normalize prob_matrix row value    
row_sum = numpy.sum(prob_matrix,axis = 1)
norm_matrix = prob_matrix / row_sum[:,numpy.newaxis]
norm_matrix = numpy.clip(norm_matrix, 1e-3, 1 - 1e-3)

numpy.log(norm_matrix)

array([[-6.90775528, -3.78199589, -6.73195975, -6.90775528, -5.56552486,
        -2.78884616, -0.42635699, -6.90775528, -6.90775528, -5.15461015,
        -5.00356989, -6.90775528, -6.90775528, -3.08884871, -6.90775528,
        -1.61807142],
       [-3.28380429, -2.89882847, -2.43762138, -6.90775528, -1.16104776,
        -2.45995733, -2.38545831, -6.90775528, -5.02833487, -5.557179  ,
        -3.48153253, -6.34563636, -3.79359041, -1.76475887, -6.90775528,
        -2.37912517],
       [-3.50465757, -5.85358084, -6.17417973, -6.90775528, -0.76944728,
        -6.13556489, -3.13341387, -6.38363783, -6.90775528, -5.9749417 ,
        -3.95703245, -5.08083061, -3.34421389, -1.11098685, -6.90775528,
        -2.78486367],
       [-2.12963129, -6.90775528, -4.96284463, -6.90775528, -1.8947917 ,
        -4.96284463, -2.47793798, -6.90775528, -6.90775528, -6.90775528,
        -3.35340672, -6.90775528, -6.90775528, -0.63211129, -6.90775528,
        -2.71155283],
       [-4.9251881 , -3.98735552, -5

In [ ]:
def viterbi(x,label,start_p,adjacency_p,emit_p):
    v= [{}]
    path = {}
    #initial t = 0,
    for y in label:
        v[0][y] = start_p[y] * emit_p[y][x[0]]
        path[y] = y

    for t in range (1,len(x)):
        v.append({})
        newpath= {}
        for tag in label:
            (prob,gues_label) = max([(v[t-1][y_0] * adjacency_p[y_0][tag]*emit_p[tag][x[t]],y_0) for y_o in label])
            v[t][tag] = prob
            newpath[tag] = path[gues_label] + [tag]
        path = newpath

    (prob,gues_label) =max([(v[len(x)-1][y],y) for y in label])
    return (prob,path[gues_label])

def emission_prob(sentence): 
    count_tag = numpy.zeros(16)
    emi = numpy.zeros(16,len(sentence))
    for i in sentence:
        # count the numbers of different tag appear in sentence
        for x,v in enumerate(num_to_rpos): 
            if pos_to_rpos[i] == v:
                count_tag[x] +=1 
    # count the numbers of each words appear in sentence
    count_word = [sentence.count(x) for x in sentence]
    
    # each word frequency to divide the each tag frequency 
    # calculate the probability of observing words from state tag
    for i in range (16):
        for j in range(len(sentence)):
            emi[i][j] = count_word[j]/count_tag[i]
    return count_v


def sentence_pos(sentence):
    """Given a sentence, as a list of tokens, this should return part of
    speech tags, as a list of strings (the codes in the rpos_desc dictionary).
    A more advanced version of token_pos that uses neighbours as well."""
    x = glove.decodes(sentence)
    x = numpy.array(x)
    
    # transition probability
    adjacency_p = numpy.log(norm_matrix)
    kitche_tag,kitsink_p = token_pos(sentence)
    
    #normalize random sink kitchen probability
    start_p = [float(i)/sum(kitsink_p) for i in len(kitsink_p)]
    
    label = len(rpos_to_num.values())
    
    emit_p = emission_prob(sentence)
    
    #calculation the prediction
    pred_y = viterbi(x,label,start_p,adjacency_p,emit_p)
        
    return pred_y 

In [ ]:
# Code to test the performance of your improved POS tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 # If you want faster feedback you can reduce this

start = time.time()
for i in range(split, len(gmb)):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess = sentence_pos(gmb[i])
    truth = gmb.pos(i)
    
    for g,t in zip(guess, truth):
        if g==pos_to_rpos[t]:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))

# 3. Named entity recognition

The next step is to identify names, that is the entities that "facts" may apply to. While training a further classifier does work (same as above, inc. dynamic programming) there would be little point in repeating the exercise. Instead, a simple rule based approach using *regular expressions* is going to be used.

You will probably want to look at the Python 3 documentation:
https://docs.python.org/3/library/re.html
There is also the how to, a tutorial:
https://docs.python.org/3/howto/regex.html


Given part of speech tagging a name can be defined as:
* An optional *determiner*, e.g. *the* (1 or none)
* An arbitrary number of *adjectives* (could be none)
* A single *noun*

In [250]:
def sentence_ner(sentence, pos):
    """Given a sentence as a list of tokens and it's part of speech tags
    this returns a list of the same length with True wherever it thinks
    there is a name."""
    
    ret = [False] * len(sentence)
    guess_lab = numpy.zeros(len(sentence))
    pos_list =''.join(pos)

    for p in re.finditer('D?',pos_list):
        if p.start()<p.end():
            guess_lab[p.start()] = guess_lab[p.start()] + 1

            
    for p in re.finditer('J*',pos_list):
        if p.start()<p.end():
            guess_lab[p.start()] = guess_lab[p.start()] + 1

    
    for p in re.finditer('N{1}',pos_list):
        if p.start()<p.end():
            guess_lab[p.start()] = guess_lab[p.start()] + 1

    for i in range (len(sentence)):
        if guess_lab[i] == 3:
            ret[i] =True 
    
    return ret

In [251]:
# Code to test the performance of the NER tagger...
correct = 0
tested = 0
pershown = 0
stop_percent = 100 

start = time.time()
for i in range(split, len(gmb)):
    percent = int(100 * (i - split) / (len(gmb) - split))
    if percent>pershown:
        pershown = percent
        print('\r{: 3d}%'.format(percent), end='')
    
    if percent>=stop_percent:
        break
    
    guess = sentence_ner(gmb[i], [pos_to_rpos[p] for p in gmb.pos(i)])
    truth = [ner!='O' for ner in gmb.ner(i)]
    
    for g,t in zip(guess, truth):
        if g==t:
            correct += 1
        tested += 1
end = time.time()

print()
print('Percentage correct = {:.1f}%'.format(100 * correct / tested))
print('  (took {:g} seconds)'.format(end - start))

 99%
Percentage correct = 84.7%
  (took 1.35122 seconds)
